In [22]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='E:\Desktop\Python\GraduationResearch2\output/vncorenlp')
# import os
# os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-18.0.1.1'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='E:\Desktop\Python\GraduationResearch2\output/vncorenlp')

VnCoreNLP model folder E:\Desktop\Python\GraduationResearch2\output/vncorenlp already exists! Please load VnCoreNLP from this folder!


In [23]:
text = rdrsegmenter.word_segment('Tổ chức nào thực hiện trách nhiệm thông báo phòng chống khi xảy ra thiên tai')
print(text)

['Tổ_chức nào thực_hiện trách_nhiệm thông_báo phòng_chống khi xảy ra thiên_tai']


In [24]:
from text_untils import *
print(word_segment('Tổ chức nào thực hiện trách nhiệm thông báo phòng chống khi xảy ra thiên tai'))

Tổ_chức nào thực_hiện trách_nhiệm thông_báo phòng_chống khi xảy ra thiên_tai


In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import cosine_similarity
import json
import pandas as pd
import torch
from text_untils import *
import faiss
import numpy as np

In [17]:
json_objects = []

with open("./data/cleaned/temp.json", 'r', encoding='utf-8') as file:
    for line in file:
        # Load each line as a JSON object
        try:
            json_object = json.loads(line)
            json_objects.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

df = pd.json_normalize(json_objects)

In [18]:
_ids = df['_id'].values
id_to_index = {i: _id for i, _id in enumerate(_ids)}
np.save('output/sbert/id_to_index.npy', id_to_index)

In [19]:
df['title'] = df['title'].apply(word_segment)
sentences = df['title'].tolist()
sentences = sentences[:1000]

In [5]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
model = AutoModel.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
model = model.to('cuda')

In [22]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
encoded_input = {key: value.to('cuda') for key, value in encoded_input.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [23]:
with torch.no_grad():
    model_output = model(**encoded_input)

In [24]:
embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = embeddings.cpu().numpy()

In [25]:
embedding_size = embeddings.shape[1]

index = faiss.IndexFlatIP(embedding_size)
index.add(embeddings)

print(f"Number of vectors in index: {index.ntotal}")

# faiss.write_index(index, 'output/sbert/vector_db.bin')

Number of vectors in index: 1000


In [18]:
index = faiss.read_index('output/sbert/embeddings_65k_index.index')

encoded_query = tokenizer(["Đi xe máy vi phạm nồng độ cồn bị phạt bao nhiêu tiền?"], padding=True, truncation=True, return_tensors='pt')
encoded_query = {key: value.to('cuda') for key, value in encoded_query.items()}

with torch.no_grad():
    model_output_query = model(**encoded_query)

embeddings_query = mean_pooling(model_output_query, encoded_query['attention_mask'])
embeddings_query = embeddings_query.cpu().numpy()
faiss.normalize_L2(embeddings_query)

In [23]:
k = 10
distances, indices = index.search(embeddings_query, k)

print(f"Indices of nearest neighbors: {indices}")
print(f"Distances of nearest neighbors: {distances}")

Indices of nearest neighbors: [[28821  7849 22104 13842 42528 10532 15779 18704  4318 15166]]
Distances of nearest neighbors: [[0.7380183  0.72621274 0.70202106 0.67678547 0.6631115  0.6554582
  0.65165067 0.64501715 0.6448065  0.64217424]]


In [24]:
id_to_index = np.load('output/sbert/id_to_index.npy', allow_pickle=True).item()

similar_ids = [id_to_index[_id] for _id in indices[0]]

In [25]:
print(similar_ids)

['6663569fc2f544363eec817c', '6663569dc2f544363eec60d5', '6663569ec2f544363eec719e', '6663569ac2f544363eec2d7c', '6663569ac2f544363eec2a7a', '6663569bc2f544363eec3692', '6663569cc2f544363eec42d8', '6663569ac2f544363eec258f', '6663568cc2f544363eec07a5', '6663568cc2f544363eec06a4']


In [27]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/bm25_65k_questions_new/docs.jsonl')

In [30]:
top_n_documents = [docs[idx] for idx in indices[0]]
print(top_n_documents)

[{'id': '6663569fc2f544363eec817c', 'title': 'lái_xe_máy nồng_độ cồn bao_nhiêu phạt'}, {'id': '6663569dc2f544363eec60d5', 'title': 'nồng_độ cồn bao_nhiêu xe_máy'}, {'id': '6663569ec2f544363eec719e', 'title': 'điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền'}, {'id': '6663569ac2f544363eec2d7c', 'title': 'lái xe_máy vi_phạm lỗi nồng_độ cồn tạm xe_máy'}, {'id': '6663569ac2f544363eec2a7a', 'title': 'nồng_độ cồn bao_nhiêu tạm xe'}, {'id': '6663569bc2f544363eec3692', 'title': 'phạt hành_vi vi_phạm nồng_độ cồn xe_đạp điện bao_nhiêu'}, {'id': '6663569cc2f544363eec42d8', 'title': 'lỗi vi_phạm nồng_độ cồn điều_khiển xe_máy tham_gia giao_thông phạt bao_nhiêu tiền'}, {'id': '6663569ac2f544363eec258f', 'title': 'điều_khiển xe_máy xe ô_tô nồng_độ cồn tạm xe'}, {'id': '6663568cc2f544363eec07a5', 'title': 'lỗi nồng_độ cồn xe_máy giam xe'}, {'id': '6663568cc2f544363eec06a4', 'title': 'điều_khiển xe_máy sử_dụng rượu_bia tham_gia giao_thông phạt bao_nhiêu tiền'}]
